In [1]:
print('helloworld')

helloworld


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [3]:
train_df=pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')
test_df=pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv')

In [4]:
def preprocess(df):


    

    
    def bool_to_int(df):
        bool_cols = df.select_dtypes(include='bool').columns
        df[bool_cols] = df[bool_cols].astype(int)
        return df


    def handle_road_type(x):
        if x == 'rural':
            return 0
        elif x == 'urban':
            return 1
        elif x == 'highway':
            return 2
        else:
            return -1


    def handle_lighting(x):
        if x== 'night':
            return 0
        elif x=='dim':
            return 1
        elif x=='daylight':
            return 2
        else: return -1

    def handle_time_of_day(x):
        if x=='morning':
            return 0
        elif x=='afternoon':
            return 1
        elif x=='evening':
            return 2
    
        else: return -1

    def handle_weather(df):
        id_col=df['id']
        dums=pd.get_dummies(df['weather'],dummy_na=False)
        remaining=df.drop(columns=['weather'])
        df = pd.concat([id_col.reset_index(drop=True), remaining.reset_index(drop=True), dums.reset_index(drop=True)], axis=1)
        return df
    df['road_type']=df['road_type'].apply(handle_road_type)
    df['time_of_day']=df['time_of_day'].apply(handle_time_of_day)
    df['lighting']=df['lighting'].apply(handle_lighting)

    df=handle_weather(df)

    df=bool_to_int(df)


    return df


In [5]:
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   id                      517754 non-null  int64  
 2   road_type               517754 non-null  int64  
 3   num_lanes               517754 non-null  int64  
 4   curvature               517754 non-null  float64
 5   speed_limit             517754 non-null  int64  
 6   lighting                517754 non-null  int64  
 7   road_signs_present      517754 non-null  int64  
 8   public_road             517754 non-null  int64  
 9   time_of_day             517754 non-null  int64  
 10  holiday                 517754 non-null  int64  
 11  school_season           517754 non-null  int64  
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
 14  clear               

In [7]:
input_features=list(train_df.columns)
input_features.remove('id')
input_features.remove('id')

input_features.remove('accident_risk')
input_features

['road_type',
 'num_lanes',
 'curvature',
 'speed_limit',
 'lighting',
 'road_signs_present',
 'public_road',
 'time_of_day',
 'holiday',
 'school_season',
 'num_reported_accidents',
 'clear',
 'foggy',
 'rainy']

In [8]:
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split

2025-10-23 17:11:39.349907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761239499.591860      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761239499.663659      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(input_features),)),
    tf.keras.layers.Dense(128, activation='elu'),
    tf.keras.layers.Dense(64, activation='elu'),
    tf.keras.layers.Dense(32, activation='elu'),
    tf.keras.layers.Dense(1, activation='elu')
])


2025-10-23 17:11:55.096995: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
# Compile the model
model.compile(optimizer='ADAM', 
              loss = tf.keras.losses.MeanAbsoluteError,
              metrics=['RootMeanSquaredError'])

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()



X_train = train_df[input_features].values
y_train = train_df['accident_risk']#.astype(int).values

X_test=test_df[input_features].values


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#449

In [12]:
model.fit(tf.expand_dims(X_train_scaled, axis=-1),y_train,epochs=50,batch_size=32,validation_split=0.2)

Epoch 1/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - RootMeanSquaredError: 0.0780 - loss: 0.0583 - val_RootMeanSquaredError: 0.0603 - val_loss: 0.0467
Epoch 2/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - RootMeanSquaredError: 0.0606 - loss: 0.0472 - val_RootMeanSquaredError: 0.0582 - val_loss: 0.0452
Epoch 3/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - RootMeanSquaredError: 0.0595 - loss: 0.0462 - val_RootMeanSquaredError: 0.0579 - val_loss: 0.0450
Epoch 4/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - RootMeanSquaredError: 0.0587 - loss: 0.0456 - val_RootMeanSquaredError: 0.0574 - val_loss: 0.0448
Epoch 5/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - RootMeanSquaredError: 0.0586 - loss: 0.0454 - val_RootMeanSquaredError: 0.0582 - val_loss: 0.0453
Epoch 6/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - RootMeanSquaredError: 0.0583 - loss: 0.0452 - val_RootMeanSquaredError: 0.0575 - val_loss: 0.0447
Epoch 7/50
12944/12944 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step -

In [13]:
test_df = test_df.loc[:,~test_df.columns.duplicated()].copy()

In [14]:
def prediction_to_kaggle_format(model, test_df):
    # Prepare features for predictions using the same input features as used in training
    # X_test = test_df[input_features].values  # Ensure input features are consistent
    risk = model.predict(X_test_scaled, verbose=0)  # Predict probabilities
    
    # Check the shape of proba_survive
    if risk.ndim > 1:
        risk = risk[:, 0]  # Flatten if necessary
    
    # Create the DataFrame for submission
    return pd.DataFrame({
        "id": test_df["id"].values,  # Ensure this is a 1D array
        "accident_risk": risk  # Convert boolean to string
    })

def make_submission(kaggle_predictions):
    path = "/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

# Generate predictions and make the submission
kaggle_predictions = prediction_to_kaggle_format(model, test_df)  # Use preprocessed test_df
make_submission(kaggle_predictions)

# Optional: Check the first few lines of the submission
!head /kaggle/working/submission.csv


Submission exported to /kaggle/working/submission.csv
id,accident_risk
517754,0.28292933
517755,0.12837091
517756,0.19100678
517757,0.30097404
517758,0.38816816
517759,0.4322775
517760,0.27069366
517761,0.20696464
517762,0.34740508
